In [162]:
import torch
from torch import nn, optim
from torch import nn
from torch.nn import functional as F

from torch_geometric import datasets
from torch_geometric.data import DataLoader
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.nn import GCNConv

import networkx as nx
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from hydra import compose, initialize

In [163]:
with initialize(version_base=None, config_path="./"):
    config = compose(config_name="model.yaml")
    print(config)

{'dataset': {'name': 'Cora', 'save_path': '.'}, 'gcn_model': {'latent_dim': 256}, 'training': {'train_size_ratio': 0.8, 'epochs': 1000, 'verbose': 10}}


In [164]:
cora = datasets.Planetoid(root=config.dataset.save_path, name=config.dataset.name, transform=NormalizeFeatures())

In [165]:
dataset = cora[0]
print(f"Number sample: {len(cora.x)}")
print(f"Number of features: {cora.num_features}")
print(f"Number of classes: {cora.num_classes}")
print(f"Number of nodes: {dataset.num_nodes}")
print(f"Number of training nodes: {dataset.train_mask.sum()}")
print(f"Is undirected? {dataset.is_undirected()}")

Number sample: 2708
Number of features: 1433
Number of classes: 7
Number of nodes: 2708
Number of training nodes: 140
Is undirected? True


In [166]:
edges = dataset.edge_index.t()

In [167]:
edges

tensor([[   0,  633],
        [   0, 1862],
        [   0, 2582],
        ...,
        [2707,  598],
        [2707, 1473],
        [2707, 2706]])

In [168]:
labels = dataset.y

In [169]:
G = nx.Graph()
G.add_edges_from(edges.tolist())

In [170]:
color_map = {}
colors = ["blue", "green", "red", "yellow", "black", "orange", "magma"]
c_map = {i : colors[i] for i in range(7)}
nodes_color = []
nodes = []
for node in G:
  nodes.append(node)
  nodes_color.append(c_map[labels[node].item()])

In [171]:
carac = pd.DataFrame({"node" : nodes, "colors" : nodes_color})

carac['colors']=pd.Categorical(carac['colors'])
carac['colors'].cat.codes

0       6
1       6
2       6
3       6
4       0
       ..
2703    5
2704    0
2705    0
2706    0
2707    0
Length: 2708, dtype: int8

In [172]:
if not os.path.exists("./cora_graph.png"):
  plt.figure(figsize=(100, 50))
  pos = nx.kamada_kawai_layout(G)
  nx.draw(G, pos, node_color=carac['colors'].cat.codes, with_labels=True)
  plt.savefig("cora_graph.png", dpi=150, bbox_inches="tight")
  plt.show()
else:
  print("./cora_graph.png")

./cora_graph.png


In [173]:
TRAIN_LIMIT = int(config.training.train_size_ratio * len(cora.x))
TEST_LIMIT = int((1 - config.training.train_size_ratio) * len(cora.x))

## Define model

In [175]:
class GNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size) -> None:
        """The module is a GNN model which is based on the GCN layer

        Parameters
        ----------
        input_size : The number of features of one node in the graph
        hidden_size : The latent dim for hidden layer
        output_size : the number of classes which is needed to classify
        """
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_size, hidden_size)
        self.conv2 = GCNConv(hidden_size, hidden_size)
        self.conv3 = GCNConv(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, X, edges_index):
        out = X
        out = self.conv1(out, edges_index)
        out = out.relu()
        out = F.dropout(out, p=0.5, training=self.training)

        out = self.conv2(out, edges_index)
        out = out.relu()
        out = F.dropout(out, p=0.5, training=self.training)

        out = self.conv3(out, edges_index)
        out = out.relu()
        out = F.dropout(out, p=0.5, training=self.training)

        out = self.out(out)
        out = F.softmax(out, dim=1)
        return out

    def get_parameters(self):
        return sum(parameter.numel() for parameter in self.parameters())

    def __str__(self):
        return (
            super(GNN, self).__str__()
            + f"\nNumber of parameters: {self.get_parameters():,}"
        )

In [176]:
LATENT_DIM = config.gcn_model.latent_dim
model = GNN(cora.num_features, LATENT_DIM, cora.num_classes)

In [177]:
print(model)

GNN(
  (conv1): GCNConv()
  (conv2): GCNConv()
  (conv3): GCNConv()
  (out): Linear(in_features=256, out_features=7, bias=True)
)
Number of parameters: 1,799


## Training

In [178]:
EPOCHS = config.training.epochs
VERBOSE = config.training.verbose
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
dataset = dataset.to(device)

In [179]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [180]:
history = {
  "epoch" : [],
  "loss" : []
}

for epoch in range(1, EPOCHS + 1):
  model.train()
  optimizer.zero_grad()
  out = model(dataset.x, dataset.edge_index)
  loss = criterion(out[dataset.train_mask], dataset.y[dataset.train_mask])
  loss.backward()
  optimizer.step()
  history["epoch"].append(epoch)
  history["loss"].append(loss.cpu().detach().item())
  
  print(f"Epoch {epoch} | Loss: {loss.item()}")

Epoch 1 | Loss: 1.9458249807357788
Epoch 2 | Loss: 1.9459444284439087
Epoch 3 | Loss: 1.9459552764892578


KeyboardInterrupt: 

In [ ]:
history_df = pd.DataFrame(history)

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(history["epoch"], history["loss"])

In [ ]:
model.eval()
out = model(dataset.x, dataset.edge_index)
pred = out.argmax(dim = 1)
test_correct = pred[dataset.test_mask] == dataset.y[dataset.test_mask]
accuracy = test_correct.sum() / dataset.test_mask.sum()
print(f"Accuracy: {accuracy}")

## Test

In [ ]:
import random
sample_index = random.randint(0, dataset.test_mask.sum()) 
sample = dataset.x[dataset.test_mask]
sns.set_theme(style="whitegrid")
plt.figure(figsize=(20, 10))
pred = model(dataset.x, dataset.edge_index)
ax = sns.barplot(x=np.array(range(7)), y=pred[sample_index].detach().cpu().numpy())
ax.set_xlabel("Classes")
ax.set_ylabel("Probability")